In [23]:
import sys
sys.path.insert(0, '..')

from ComputeC import computeC
from Process_gene_data import process_gene_data
import numpy as np
import json
import matplotlib.pyplot as plt
from brokenaxes import brokenaxes
from matplotlib.ticker import FixedLocator
from matplotlib.lines import Line2D

Data one

In [2]:
# 1
filename = '/Users/gyh/Documents/github/TCCN/Gene_Data/example_1.txt'
arrays, gene_dict = process_gene_data(filename)
print(arrays)
print(gene_dict)

[[1, 2, 3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24], [1, 2, 3, 4, 5, 6], [4, 5, 8, 9, 10, 11], [9, 10, 12, 13, 14, 15], [22, 23, 25, 26]]
{1: 919, 2: 184, 3: 179, 4: 176, 5: 106, 6: 945, 7: 82, 8: 91, 9: 107, 10: 90, 11: 81, 12: 157, 13: 107, 14: 103, 15: 85, 16: 135, 17: 96, 18: 138, 19: 117, 20: 166, 21: 386, 22: 180, 23: 79, 24: 216, 25: 216, 26: 95}


In [ ]:
# Calculate condition number
sums = [sum(gene_dict[key] for key in sublist) for sublist in arrays]

# Find the sublist with maximum sum
max_sum = max(sums)
max_readlengh = max_sum

maxc = []
k = 0
# Generate readlengh range
readlengh_range = np.linspace(100, 2501, 25, endpoint=True, dtype=int)

for readlengh in readlengh_range:
    C = computeC(arrays, readlengh, gene_dict)
    eigenvalues, eigenvectors = np.linalg.eig(C.T @ C)
    max_element = max(eigenvalues)/min(eigenvalues)
    maxc.append(max_element)

# Import data from figure 1
with open('Figure_2_data/data_1.json', 'r') as json_file:
    data = json.load(json_file)

median_values_mutiple = [np.median(box) for box in data]

In [ ]:
# Simulate unique mapping scenario
np.random.seed(1)  # Set fixed seed
w = np.random.dirichlet(np.ones(len(arrays)), size=1)[0]
np.random.seed(None)  # Cancel fixed seed, restore randomness

# Set number of samples
n = 800

# Set number of repetitions
times = 100

# Determine length range
readlengh_range = np.arange(100, 2501, 100)

# Determine unique mapping error
left_all = []

for readlengh in readlengh_range:   
    left_temp = []
    
    length = []
    for array in arrays:
        length_array = sum([gene_dict[key] for key in array])
        length.append(length_array)
        
    if any(l <= readlengh for l in length):
        indices_to_remove = [i for i, l in enumerate(length) if l <= readlengh]
        w_new = np.delete(w, indices_to_remove)
        w_new = w_new / np.sum(w_new)  # Redistribute proportions
    else:
        w_new = w
    
    C = computeC(arrays, readlengh, gene_dict)

    for i in range(times):

        # Calculate exact P
        P = C@w_new.T

        Delta_P = []
        # Calculate fluctuation values based on sampling number n and values in p
        def generate_normal_distribution(value, n):
            variance = value * (1 - value) / n
            std_deviation = np.sqrt(variance)
            return np.random.normal(0, std_deviation)

        for value in P:
            Delta_P.append(generate_normal_distribution(value, n))

        Delta_P_norm = np.array(Delta_P)
        #Delta_P_norm1 = Delta_P_norm[0]
        # Calculate normalized P_hat
        P_hat = (P+Delta_P_norm)/np.sum(P+Delta_P_norm)

        # Inverse solve for true w (multiple mapping)
        w_hat = np.linalg.inv(C.T@C)@C.T@(P_hat)
        Delta_w = np.linalg.norm(w_hat - w_new, ord=2)
        
        # Solve for unique mapping error
        # First calculate sampling proportions
        num = []
        for i in range(C.shape[1]):
            num.append(n*P_hat[i])
        
        sum_num = sum(num)
        if sum_num == 0:
            w_hat_unique = [0 for i in num]
            print('Cannot calculate')
        else:
            w_hat_unique = [i/sum_num for i in num]    
            
            # Calculate error for unique mapping
            Delta_w_unique = np.linalg.norm(np.array(w_hat_unique) - w_new, ord=2) 
    
        left_temp.append(Delta_w_unique)
    left_all.append(left_temp)

median_values = [np.median(box) for box in left_all]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from brokenaxes import brokenaxes
from matplotlib.ticker import ScalarFormatter, FixedLocator
from matplotlib.lines import Line2D

readlengh_range = np.arange(100, 2501, 100)

# Create main plot
fig = plt.figure(figsize=(15, 12), dpi = 600)

# Use brokenaxes to create plot with truncated y-axis
bax = brokenaxes(ylims=((0, 5), (11995, 12000)), hspace=.3, d=0.005)  # Adjust ylims and hspace to fit your data

# Plot data
line1 = bax.plot(readlengh_range, maxc, 'o-', markersize=10, color='gray', label='Condition Number', linewidth=4)

ax2 = bax.axs[1].twinx()
ax3 = bax.axs[0].twinx()

ax2.set_ylim(0, 0.2)
ax3.set_ylim(0.5, 0.7)
ax3.set_yticks([0.5, 0.7])
#ax3.set_ylim(['0.7'])

line2, = ax2.plot(readlengh_range, median_values, 'b-d', markersize=10, label='Unique Mapping', linewidth=4)
line3, = ax2.plot(readlengh_range, median_values_mutiple, 'r^-', markersize=10, label='Multiple Mapping', linewidth=4)
# ax2.set_ylabel('Error', fontsize=45, labelpad=45, rotation=270)

# Create proxy artist
proxy = [Line2D([0], [0], color='gray', marker='o', linestyle='-', markersize=10, label='Condition Number', linewidth=4),
         Line2D([0], [0], color='blue', marker='d', linestyle='-', markersize=10, label='Unique Mapping', linewidth=4),
         Line2D([0], [0], color='r', marker='^', linestyle='-', markersize=10, label='Multiple Mapping', linewidth=4)]

# Add grid lines
bax.grid(True, linestyle='--', alpha=1.0, linewidth=2.0)

grid_positions = [100, 500, 900, 1300, 1700, 2100, 2500]  # Set vertical grid line positions
for ax in bax.axs:
    ax.yaxis.set_major_formatter(ScalarFormatter())
    ax.yaxis.get_major_formatter().set_scientific(False)
    ax.yaxis.get_major_formatter().set_useOffset(False)
    ax.xaxis.set_major_locator(FixedLocator(grid_positions))  # Set exact positions for vertical grid lines
    ax.grid(True, which='major', axis='x', linestyle='--', alpha=1.0, linewidth=2.0)
    
# Add right and top borders
for ax in bax.axs:
    ax.spines['right'].set_visible(True)
    ax.spines['top'].set_visible(True)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.tick_params(axis='x', width=4, colors='black', labelsize=30, labelcolor='black')
    ax.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')

ax2.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')
ax3.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')
# Only add horizontal lines on top and right
bax.axs[1].spines['top'].set_visible(False)  # Hide bottom border

# Show plot
plt.show()

Data_two

In [ ]:
# 2
filename = '/Users/gyh/Documents/github/TCCN/Gene_Data/example_2.txt'
arrays, gene_dict = process_gene_data(filename)
# Calculate condition number
sums = [sum(gene_dict[key] for key in sublist) for sublist in arrays]

# Find the sublist with maximum sum
max_sum = max(sums)
max_readlengh = max_sum

maxc = []
k = 0
# Generate readlengh range
readlengh_range = np.linspace(100, 2501, 25, endpoint=True, dtype=int)

for readlengh in readlengh_range:
    C = computeC(arrays, readlengh, gene_dict)
    eigenvalues, eigenvectors = np.linalg.eig(C.T @ C)
    max_element = max(eigenvalues)/min(eigenvalues)
    maxc.append(max_element)

# Import data from figure 1
with open('Figure_2_data/data_2.json', 'r') as json_file:
    data = json.load(json_file)

median_values_mutiple = [np.median(box) for box in data]
# Simulate unique mapping scenario
np.random.seed(1)  # Set fixed seed
w = np.random.dirichlet(np.ones(len(arrays)), size=1)[0]
np.random.seed(None)  # Cancel fixed seed, restore randomness

# Set number of samples
n = 800

# Set number of repetitions
times = 100

# Determine length range
readlengh_range = np.arange(100, 2501, 100)

# Determine unique mapping error
left_all = []

for readlengh in readlengh_range:   
    left_temp = []
    
    length = []
    for array in arrays:
        length_array = sum([gene_dict[key] for key in array])
        length.append(length_array)
        
    if any(l <= readlengh for l in length):
        indices_to_remove = [i for i, l in enumerate(length) if l <= readlengh]
        w_new = np.delete(w, indices_to_remove)
        w_new = w_new / np.sum(w_new)  # Redistribute proportions
    else:
        w_new = w
    
    C = computeC(arrays, readlengh, gene_dict)

    for i in range(times):

        # Calculate exact P
        P = C@w_new.T

        Delta_P = []
        # Calculate fluctuation values based on sampling number n and values in p
        def generate_normal_distribution(value, n):
            variance = value * (1 - value) / n
            std_deviation = np.sqrt(variance)
            return np.random.normal(0, std_deviation)

        for value in P:
            Delta_P.append(generate_normal_distribution(value, n))

        Delta_P_norm = np.array(Delta_P)
        #Delta_P_norm1 = Delta_P_norm[0]
        # Calculate normalized P_hat
        P_hat = (P+Delta_P_norm)/np.sum(P+Delta_P_norm)

        # Inverse solve for true w (multiple mapping)
        w_hat = np.linalg.inv(C.T@C)@C.T@(P_hat)
        Delta_w = np.linalg.norm(w_hat - w_new, ord=2)
        
        # Solve for unique mapping error
        # First calculate sampling proportions
        num = []
        for i in range(C.shape[1]):
            num.append(n*P_hat[i])
        
        sum_num = sum(num)
        if sum_num == 0:
            w_hat_unique = [0 for i in num]
            print('Cannot calculate')
        else:
            w_hat_unique = [i/sum_num for i in num]    
            
            # Calculate error for unique mapping
            Delta_w_unique = np.linalg.norm(np.array(w_hat_unique) - w_new, ord=2) 
    
        left_temp.append(Delta_w_unique)
    left_all.append(left_temp)

median_values = [np.median(box) for box in left_all]

In [ ]:
# Assume maxc is your data, generate some simulated data here
readlengh_range = np.arange(100, 2501, 100)

# Create main plot
fig = plt.figure(figsize=(15, 12), dpi = 600)

# Use brokenaxes to create plot with truncated y-axis
bax = brokenaxes(ylims=((0, 900), (11965, 12000)), hspace=.2, d=0.005)  # Adjust ylims and hspace to fit your data

# Plot data
line1 = bax.plot(readlengh_range, maxc, 'o-', markersize=10, color='gray', label='Condition Number', linewidth=4)

ax2 = bax.axs[1].twinx()
ax3 = bax.axs[0].twinx()

ax2.set_ylim(0, 0.5)
ax3.set_ylim(0.5, 0.7)
ax3.set_yticks([0.7])
#ax3.set_ylim(['0.7'])

line2, = ax2.plot(readlengh_range, median_values, 'b-d', markersize=10, label='Unique Mapping', linewidth=4)
line3, = ax2.plot(readlengh_range, median_values_mutiple, 'r^-', markersize=10, label='Multiple Mapping', linewidth=4)
# ax2.set_ylabel('Error', fontsize=45, labelpad=45, rotation=270)

# Create proxy artist
proxy = [Line2D([0], [0], color='gray', marker='o', linestyle='-', markersize=10, label='Condition Number', linewidth=4),
         Line2D([0], [0], color='blue', marker='d', linestyle='-', markersize=10, label='Unique Mapping', linewidth=4),
         Line2D([0], [0], color='r', marker='^', linestyle='-', markersize=10, label='Multiple Mapping', linewidth=4)]

# Add grid lines
bax.grid(True, linestyle='--', alpha=1.0, linewidth=2.0)

grid_positions = [100, 500, 900, 1300, 1700, 2100, 2500]  # Set vertical grid line positions
for ax in bax.axs:
    ax.xaxis.set_major_locator(FixedLocator(grid_positions))  # Set exact positions for vertical grid lines
    ax.grid(True, which='major', axis='x', linestyle='--', alpha=1.0, linewidth=2.0)
    
# Add right and top borders
for ax in bax.axs:
    ax.spines['right'].set_visible(True)
    ax.spines['top'].set_visible(True)
    ax.spines['bottom'].set_linewidth(2)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_color('black')
    ax.spines['left'].set_color('black')
    ax.tick_params(axis='x', width=4, colors='black', labelsize=30, labelcolor='black')
    ax.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')

ax2.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')
ax3.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')
# Only add horizontal lines on top and right
bax.axs[1].spines['top'].set_visible(False)  # Hide bottom border

# Show plot
plt.show()

Data_three

In [ ]:
# 3
filename = '/Users/gyh/Documents/github/TCCN/Gene_Data/example_3.txt'
arrays, gene_dict = process_gene_data(filename)
print(arrays)
print(gene_dict)
# Calculate condition number
sums = [sum(gene_dict[key] for key in sublist) for sublist in arrays]

# Find the sublist with maximum sum
max_sum = max(sums)
max_readlengh = max_sum

maxc = []
k = 0
# Generate readlengh range
readlengh_range = np.linspace(100, 2501, 25, endpoint=True, dtype=int)

for readlengh in readlengh_range:
    C = computeC(arrays, readlengh, gene_dict)
    eigenvalues, eigenvectors = np.linalg.eig(C.T @ C)
    max_element = max(eigenvalues)/min(eigenvalues)
    maxc.append(max_element)

# Import data from figure 1
with open('Figure_2_data/data_3.json', 'r') as json_file:
    data = json.load(json_file)

median_values_mutiple = [np.median(box) for box in data]
# Simulate unique mapping scenario
np.random.seed(1)  # Set fixed seed
w = np.random.dirichlet(np.ones(len(arrays)), size=1)[0]
np.random.seed(None)  # Cancel fixed seed, restore randomness

# Set number of samples
n = 400

# Set number of repetitions
times = 100

# Determine length range
readlengh_range = np.arange(100, 2501, 100)

# Determine unique mapping error
left_all = []

for readlengh in readlengh_range:   
    left_temp = []
    
    length = []
    for array in arrays:
        length_array = sum([gene_dict[key] for key in array])
        length.append(length_array)
        
    if any(l <= readlengh for l in length):
        indices_to_remove = [i for i, l in enumerate(length) if l <= readlengh]
        w_new = np.delete(w, indices_to_remove)
        w_new = w_new / np.sum(w_new)  # Redistribute proportions
    else:
        w_new = w
    
    C = computeC(arrays, readlengh, gene_dict)

    for i in range(times):

        # Calculate exact P
        P = C@w_new.T

        Delta_P = []
        # Calculate fluctuation values based on sampling number n and values in p
        def generate_normal_distribution(value, n):
            variance = value * (1 - value) / n
            std_deviation = np.sqrt(variance)
            return np.random.normal(0, std_deviation)

        for value in P:
            Delta_P.append(generate_normal_distribution(value, n))

        Delta_P_norm = np.array(Delta_P)
        #Delta_P_norm1 = Delta_P_norm[0]
        # Calculate normalized P_hat
        P_hat = (P+Delta_P_norm)/np.sum(P+Delta_P_norm)

        # Inverse solve for true w (multiple mapping)
        w_hat = np.linalg.inv(C.T@C)@C.T@(P_hat)
        Delta_w = np.linalg.norm(w_hat - w_new, ord=2)
        
        # Solve for unique mapping error
        # First calculate sampling proportions
        num = []
        for i in range(C.shape[1]):
            num.append(n*P_hat[i])
        
        sum_num = sum(num)
        if sum_num == 0:
            w_hat_unique = [0 for i in num]
            print('Cannot calculate')
        else:
            w_hat_unique = [i/sum_num for i in num]    
            
            # Calculate error for unique mapping
            Delta_w_unique = np.linalg.norm(np.array(w_hat_unique) - w_new, ord=2) 
    
        left_temp.append(Delta_w_unique)
    left_all.append(left_temp)

median_values = [np.median(box) for box in left_all]

[[1, 2, 3, 5, 10], [1, 2, 3, 10], [1, 2, 3, 5, 8], [1, 2, 3, 5, 7], [3, 5, 10], [3, 5, 11], [3, 5, 6], [3, 4, 5, 9]]
{1: 2559, 2: 153, 3: 462, 4: 139, 5: 56, 6: 337, 7: 29, 8: 172, 9: 161, 10: 315, 11: 210}


In [ ]:
from matplotlib.ticker import MultipleLocator, FixedLocator

# Create main plot
fig, ax1 = plt.subplots(figsize=(15, 12), dpi = 600)

fig=plt.gcf()

readlengh_range = np.arange(100, 2501, 100)

# Plot first set of data (needs truncation handling)
line1, = ax1.plot(readlengh_range, maxc, 'o-', markersize=10, color = 'gray', label='Condition Number', linewidth=4)
#ax1.set_ylim(0, 20)
# ax1.set_xlabel('Read Length', fontsize=45, labelpad=15)
# ax1.set_ylabel('Condition Number', fontsize=45, labelpad=9)

ax2 = ax1.twinx()
line2, = ax2.plot(readlengh_range, median_values, 'b-d', markersize=10, label='Unique Mapping', linewidth=4)
line3, = ax2.plot(readlengh_range, median_values_mutiple, 'r^-', markersize=10, label='Multiple Mapping', linewidth=4)
# ax2.set_ylabel('Error', fontsize=45, labelpad=45, rotation=270)

# Adjust inset window position and size
inset_ax = fig.add_axes([0.45, 0.7, 0.15, 0.15])  # [left, bottom, width, height]

# Plot high value points of first data set in inset
inset_ax.plot(readlengh_range, maxc, 'o-', color = 'gray',markersize=10, linewidth=4)
inset_ax.set_ylim(40000, 50000)

# Hide inset x-axis information
inset_ax.set_xticks([])  # Hide x-axis ticks
inset_ax.set_xlabel('')  # Hide x-axis label

# Set y-axis range
ax1.set_ylim(0, 12000)
ax2.set_ylim(0, 0.7)

grid_positions = [100, 500, 900, 1300, 1700, 2100, 2500]  # Set vertical grid line positions
ax1.xaxis.set_major_locator(FixedLocator(grid_positions))  # Set exact positions for vertical grid lines
ax1.grid(True, which='major', axis='x', linestyle='--', alpha=1.0, linewidth=2.0)

# Add grid lines
inset_ax.grid(True, linestyle='--', alpha=1.0, linewidth=2.0)
ax1.grid(True, linestyle='--', alpha=1.0, linewidth=2.0)


ax1.spines['bottom'].set_linewidth(2)
ax1.spines['left'].set_linewidth(2)
ax1.spines['bottom'].set_color('black')
ax1.spines['left'].set_color('black')
ax1.tick_params(axis='x', width=4, colors='black', labelsize=30, labelcolor='black')
ax1.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')
ax2.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')

inset_ax.tick_params(axis='x', width=4, colors='black', labelsize=30, labelcolor='black')
inset_ax.tick_params(axis='y', width=4, colors='black', labelsize=30, labelcolor='black')

lines = [line1, line2, line3]
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc='upper right', fontsize=25, frameon=True, bbox_to_anchor=(1.01, 1.25))

# Show plot
plt.show()